In [117]:
import solana.system_program as sp
from solana.publickey import PublicKey
from solana.keypair import Keypair
from solana.account import Account
from solana.rpc.api import Client
from solana.rpc.async_api import AsyncClient
from solana.transaction import Transaction, TransactionInstruction, AccountMeta
from spl.token.async_client import AsyncToken
from spl.token.constants import TOKEN_PROGRAM_ID
from solana.rpc.types import TxOpts

from solana.transaction import Transaction, TransactionInstruction
from spl.token.instructions import (
    initialize_mint,
    InitializeMintParams,
    initialize_account,
    InitializeAccountParams,
    mint_to_checked,
    MintToCheckedParams,
    get_associated_token_address,
    create_associated_token_account,
    transfer_checked,
    TransferCheckedParams,
)


In [32]:
cli = AsyncClient('https://api.devnet.solana.com')

In [33]:
kp_mint = Keypair.generate()
kp_payer = Keypair.generate()
kp_escrow = Keypair.generate()
kp_reciever = Keypair.generate()
kp_fee_payer = Keypair.generate()


In [135]:
print(kp_mint.public_key)
print(kp_payer.public_key)
print(kp_escrow.public_key)
print(kp_reciever.public_key)
print(kp_fee_payer.public_key)

4L9ihc7p7VepgVyKCmSPcQwWPzQB6STVAVpn7gS61hku
DDWJiqAoBEQUZRxzS2VPpgWidkhdaStVGa6oxYkC2BMw
3qxjtthxJx9DNzssKSyskgyzJYXP9bwjJVMcvosriSgo
4sxiJJHi7c3tbq7yEnkbRVtBc3mdtV6AncwsyAr9zPAG
F4yF9fdmbcNgiCY4rcp7M8zWSYdGQpa8VYvS6iB9o4jE


In [34]:
kp_mint.public_key

4L9ihc7p7VepgVyKCmSPcQwWPzQB6STVAVpn7gS61hku

In [74]:
await cli.request_airdrop(kp_payer.public_key, 100000000)

{'jsonrpc': '2.0',
 'result': 'qHto1efHLkPi8Vw97rRxt3nDAPV63XhHDi2y15y9esmqh8NUY4Hp4uAHdyNK67sju7ZecpfXrtTP6VFQdftHjmt',
 'id': 89}

In [76]:
await cli.request_airdrop(kp_mint.public_key, 100000000)

{'jsonrpc': '2.0',
 'result': '5qBctn7jysVJgCinbaNDEzaT7Rer7ehMYjdhuHzp6CZbnAWF2VPoLeW48zYNjrwTjZ4SCBMyAxQ4vbrYLrKLxTaY',
 'id': 91}

In [130]:
await cli.request_airdrop(kp_fee_payer.public_key, 100000000)

{'jsonrpc': '2.0',
 'result': 'KXrDhm63DjW6NpNM9yFMyqmtrjMj45vLpkGWPYLKSKe2VweeGA8znZboPKPiEAEQtnKjnDHyVGcsdKgbTTpSxSE',
 'id': 168}

In [80]:
await cli.get_balance(kp_mint.public_key)

{'jsonrpc': '2.0',
 'result': {'context': {'slot': 137725624}, 'value': 100980000},
 'id': 95}

In [81]:
token = await AsyncToken.create_mint(cli, kp_mint, kp_mint.public_key, 6, TOKEN_PROGRAM_ID, kp_mint.public_key)
token

In [82]:
await cli.get_balance(kp_mint.public_key)

{'jsonrpc': '2.0',
 'result': {'context': {'slot': 137725728}, 'value': 99508400},
 'id': 125}

In [83]:
token.pubkey

ECgBQi1xz9oDAQKzp7rUCjcEExkEVCwoPupKfpiKzgvi

In [84]:
await token.get_mint_info()

MintInfo(mint_authority=4L9ihc7p7VepgVyKCmSPcQwWPzQB6STVAVpn7gS61hku, supply=0, decimals=6, is_initialized=True, freeze_authority=4L9ihc7p7VepgVyKCmSPcQwWPzQB6STVAVpn7gS61hku)

In [105]:
print(kp_mint.public_key)
print(token.pubkey)
type(token.pubkey)

4L9ihc7p7VepgVyKCmSPcQwWPzQB6STVAVpn7gS61hku
ECgBQi1xz9oDAQKzp7rUCjcEExkEVCwoPupKfpiKzgvi


solana.publickey.PublicKey

In [104]:
kp_mint_tokaddr = get_associated_token_address(kp_mint.public_key, token.pubkey)
kp_mint_tokaddr

7yfkb1K6cj3moWpofCSmgQJpqiB3ZK3bDJ4XVZEMUriw

In [98]:
# , opts={'feePayer': kp_mint}
# opts=TxOpts(skip_confirmation=False)
# opts=TxOpts(feePayer=kp_mint)
# await token.mint_to_checked(kp_payer.public_key, kp_mint.public_key, 1000, 6)

txn = Transaction(fee_payer=kp_mint.public_key)
txn.add(
    mint_to_checked(
        MintToCheckedParams(
            TOKEN_PROGRAM_ID,
            token.pubkey,
            kp_mint_tokaddr,
            kp_mint.public_key,
            10,
            6,
        )
    )
)
signers = [kp_mint]
await cli.send_transaction(txn, *signers)



RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: invalid account data for instruction', 'data': {'accounts': None, 'err': {'InstructionError': [0, 'InvalidAccountData']}, 'logs': ['Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [1]', 'Program log: Instruction: MintToChecked', 'Program log: Error: InvalidAccountData', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 1601 of 1400000 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA failed: invalid account data for instruction'], 'unitsConsumed': 0}}

In [108]:

txn = Transaction(fee_payer=kp_mint.public_key)
txn.add(
        create_associated_token_account(kp_mint.public_key, kp_mint.public_key, token.pubkey)
    )
signers = [kp_mint]
await cli.send_transaction(txn, *signers)



RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: custom program error: 0x0', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'Custom': 0}]}, 'logs': ['Program ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL invoke [1]', 'Program log: Create', 'Program 11111111111111111111111111111111 invoke [2]', 'Allocate: account Address { address: 7yfkb1K6cj3moWpofCSmgQJpqiB3ZK3bDJ4XVZEMUriw, base: None } already in use', 'Program 11111111111111111111111111111111 failed: custom program error: 0x0', 'Program ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL consumed 14022 of 1400000 compute units', 'Program ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL failed: custom program error: 0x0'], 'unitsConsumed': 0}}

In [109]:
txn = Transaction(fee_payer=kp_mint.public_key)

txn.add(
    mint_to_checked(
        MintToCheckedParams(
            TOKEN_PROGRAM_ID,
            token.pubkey,
            kp_mint_tokaddr,
            kp_mint.public_key,
            10,
            6,
        )
    )
)
signers = [kp_mint]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': '4pyRT2NNbPPqhxJCMGzzEXh397hvGGYY9qKwv7WZzornfK2YexiEK1Z7JK4mTxmzjKeu8AiUaUWXcwJJJHEi6QSm',
 'id': 145}

In [110]:

txn = Transaction(fee_payer=kp_mint.public_key)
txn.add(
        create_associated_token_account(kp_mint.public_key, kp_payer.public_key, token.pubkey)
    )
signers = [kp_mint]
await cli.send_transaction(txn, *signers)



RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: custom program error: 0x0', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'Custom': 0}]}, 'logs': ['Program ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL invoke [1]', 'Program log: Create', 'Program 11111111111111111111111111111111 invoke [2]', 'Allocate: account Address { address: 3pK1T4VdqsXbftbrUip3Ew4RJqVkmhwaMyiBJQBYQFLv, base: None } already in use', 'Program 11111111111111111111111111111111 failed: custom program error: 0x0', 'Program ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL consumed 6608 of 1400000 compute units', 'Program ATokenGPvbdGVxr1b2hvZbsiqW5xWH25efTNsLJA8knL failed: custom program error: 0x0'], 'unitsConsumed': 0}}

In [111]:
kp_payer_tokaddr = get_associated_token_address(kp_payer.public_key, token.pubkey)


txn = Transaction(fee_payer=kp_mint.public_key)

txn.add(
    mint_to_checked(
        MintToCheckedParams(
            TOKEN_PROGRAM_ID,
            token.pubkey,
            kp_payer_tokaddr,
            kp_mint.public_key,
            10,
            6,
        )
    )
)
signers = [kp_mint]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': '66BEhqDQLkBcQh9T58DoGGzNhGJ1gPpvMHoNsH6Ha4BwqE7DPWiqNWbw37hTPrDquLxJCZHAE5LNXE2oiDoqZ8DH',
 'id': 149}

In [112]:
# here we can see the balance

await token.get_accounts(kp_payer.public_key)


{'jsonrpc': '2.0',
 'result': {'context': {'slot': 137737078},
  'value': [{'account': {'data': {'parsed': {'info': {'isNative': False,
        'mint': 'ECgBQi1xz9oDAQKzp7rUCjcEExkEVCwoPupKfpiKzgvi',
        'owner': 'DDWJiqAoBEQUZRxzS2VPpgWidkhdaStVGa6oxYkC2BMw',
        'state': 'initialized',
        'tokenAmount': {'amount': '10',
         'decimals': 6,
         'uiAmount': 1e-05,
         'uiAmountString': '0.00001'}},
       'type': 'account'},
      'program': 'spl-token',
      'space': 165},
     'executable': False,
     'lamports': 2039280,
     'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
     'rentEpoch': 318},
    'pubkey': '3pK1T4VdqsXbftbrUip3Ew4RJqVkmhwaMyiBJQBYQFLv'}]},
 'id': 150}

In [113]:
txn = Transaction(fee_payer=kp_mint.public_key)
txn.add(
        create_associated_token_account(kp_mint.public_key, kp_reciever.public_key, token.pubkey)
    )
signers = [kp_mint]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': 'kUVtazjgyi4ktDAhtJj3b4z6RE46hBw6CEDRBPFamhRPbun6fPd8qzVE8xSft9A7D3mFcqM2Y5NqVjsoD8HNKX5',
 'id': 152}

In [114]:
txn = Transaction(fee_payer=kp_mint.public_key)
txn.add(
        create_associated_token_account(kp_mint.public_key, kp_escrow.public_key, token.pubkey)
    )
signers = [kp_mint]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': '2szSyfmnfxv2m1JdNXSG6vYTBNL4hxyWPT8xUZhW2jNKuoArC9gpKu47fAPyLbzMSir6t5AMauSRN7E5anpaMJBJ',
 'id': 154}

In [123]:
kp_payer.public_key

DDWJiqAoBEQUZRxzS2VPpgWidkhdaStVGa6oxYkC2BMw

In [125]:

txn = Transaction(fee_payer=kp_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=12,
            decimals=6
        )
    )
)
signers = [kp_payer]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': '3R9FfyBbXeTH5hE8yaXU4NTuiBcyZEj2AqBs9UJu9AtyKANGZJ5iXGSJysMqMh6RxQopeq6LgKK6hSXooMHQtXnp',
 'id': 160}

In [126]:

txn = Transaction(fee_payer=kp_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=12,
            decimals=6
        )
    )
)
signers = [kp_payer]
await cli.send_transaction(txn, *signers)

# failed — too big amount

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: custom program error: 0x1', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'Custom': 1}]}, 'logs': ['Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [1]', 'Program log: Instruction: TransferChecked', 'Program log: Error: insufficient funds', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 2827 of 1400000 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA failed: custom program error: 0x1'], 'unitsConsumed': 0}}

In [128]:

txn = Transaction(fee_payer=kp_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_reciever.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)


signers = [kp_payer]
await cli.send_transaction(txn, *signers)



{'jsonrpc': '2.0',
 'result': '2UCcqK3YLgFDcSQLXPJxzyHNwtcNcN7CU1VLZpAbMAb4KNPL84EwcHjdkCGDLBxeTjx6SFmp8nspHCgYosd5f33W',
 'id': 165}

In [137]:
# fee payer and owner are different

txn = Transaction(fee_payer=kp_fee_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)
signers = [kp_fee_payer, kp_payer]
await cli.send_transaction(txn, *signers)



{'jsonrpc': '2.0',
 'result': '2MurL5W6UWovyN99feGoyLobBWDCkJwqcyjKd2zXsMdniTnFtMbEGKcSVUpvi6ChRf2AjHtUyviQKPr1gcXeYGDB',
 'id': 178}

In [134]:

txn = Transaction(fee_payer=kp_fee_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_reciever.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)


signers = [kp_fee_payer, kp_payer]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': '5pxMLsXicC7scvE1b7cJcvwU3okRdW6vxs5s1LzTnmutdBavhsCaGbKSxzd8DCUm9n8aTpHNp2uxvd12HN5QUzME',
 'id': 174}